In [1]:
# install sparqlwrapper package
!pip install -q sparqlwrapper

     |████████████████████████████████| 407 kB 31.4 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 


In [2]:
# SPARQLWrapper execute SPARQLE queries and to import result into python

In [4]:
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3, RDF

In [5]:
# use dbpedia as our SPARQL endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [15]:
# SPARQL query to be executed
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dc:  <http://purl.org/dc/elements/1.1/>
Select distinct ?birthdate ?thumbnail ?scientist ?name ?description  WHERE {
?scientist rdf:type dbo:Scientist ;
        dbo:birthDate ?birthdate ;
        rdfs:label ?name ;
        rdfs:comment ?description 
   FILTER ((lang(?name)="en")&&(lang(?description)="en")&&(STRLEN(STR(?birthdate))>6)&&(SUBSTR(STR(?birthdate),6)=SUBSTR(STR(bif:curdate('')),6))) .
 OPTIONAL { ?scientist dbo:thumbnail ?thumbnail . }
} ORDER BY ?birthdate
""")


In [17]:
sparql.setReturnFormat(JSON) # return format is json
results = sparql.query().convert() #execute SPARQL query and write result to "results"

In [18]:
results

{'head': {'link': [],
  'vars': ['birthdate', 'thumbnail', 'scientist', 'name', 'description']},
 'results': {'bindings': [{'birthdate': {'datatype': 'http://www.w3.org/2001/XMLSchema#date',
     'type': 'typed-literal',
     'value': '1666-11-01'},
    'description': {'type': 'literal',
     'value': 'James Sherard (1 November 1666 – 12 February 1738) was an English apothecary, botanist, and amateur musician.',
     'xml:lang': 'en'},
    'name': {'type': 'literal', 'value': 'James Sherard', 'xml:lang': 'en'},
    'scientist': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/James_Sherard'}},
   {'birthdate': {'datatype': 'http://www.w3.org/2001/XMLSchema#date',
     'type': 'typed-literal',
     'value': '1724-11-01'},
    'description': {'type': 'literal',
     'value': 'Mihalj Šilobod Bolšić (1 November 1724 – 4 April 1787) was a Croatian mathematician and musical theorist primarily known for writing the first Croatian arithmetics textbook Arithmetika Horvatzka (published

In [22]:
# create HTML output
print("<html><head><title>Scientific Birthdays of Today</title></head>") 
# extract weekday %A /Month %B /day of month 5d by formatting todays date accordingly
date = datetime.today().strftime("5A %B %d")
print('<body><h1>Scientific Birthdays of {}</h1>'.format(date))
print("<ul>")

for result in results["results"]["bindings"]:
  if ("scientist" in result):
    #create a wikipedia link
    wikiurl = "http://en.wikipedia.org/wiki/" + result["scientist"]["value"].split("/")[-1]
  else:
    wikiurl = "NONE"
  if ("name" in result):
    name = result["name"]["value"]
  else:
    name ="NONE"
  if ("birthdate" in result):
    birthdate = result["birthdate"]["value"]
  else:
    birthdate ="NONE"
  if ("description" in result):
    description = result["description"]["value"]
  else:
    description = ""
  if ("thumbnail" in result):
    pic = result["thumbnail"]["value"]
  else:
    pic = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Question_mark2.svg/71px-Question_mark2.svg.png"        	
  # parse date as datetime
  dt = datetime.strptime(birthdate , "%Y-%m-%d")
  print('<li><b>{}</b> -- <img src="{}" height="60px"> <a href="{}">{}</a>, {} </li>'.format(dt.year, pic.replace("300", "60"), wikiurl, name, description))

print('</ul>')
print('</body></html>')
   

<html><head><title>Scientific Birthdays of Today</title></head>
<body><h1>Scientific Birthdays of 5A November 01</h1>
<ul>
<li><b>1666</b> -- <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Question_mark2.svg/71px-Question_mark2.svg.png" height="60px"> <a href="http://en.wikipedia.org/wiki/James_Sherard">James Sherard</a>, James Sherard (1 November 1666 – 12 February 1738) was an English apothecary, botanist, and amateur musician. </li>
<li><b>1724</b> -- <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Question_mark2.svg/71px-Question_mark2.svg.png" height="60px"> <a href="http://en.wikipedia.org/wiki/Mihalj_Šilobod_Bolšić">Mihalj Šilobod Bolšić</a>, Mihalj Šilobod Bolšić (1 November 1724 – 4 April 1787) was a Croatian mathematician and musical theorist primarily known for writing the first Croatian arithmetics textbook Arithmetika Horvatzka (published in Zagreb, 1758). </li>
<li><b>1805</b> -- <img src="http://commons.wikimedia.org/wiki/Special:

In [29]:
# write output into file on local computer
from google.colab import files

with open("birthday.html","w") as f:
  #create html output
  f.write("<html><head><title>Scientific Birthdays of Today</title></head>")
  #extract weekday %A /month %B /day of the month %d  by formatting todays date accordingly
  date = datetime.today().strftime("%A %B %d")
  f.write("<body><h1>Scientific Birthdays of {}</h1>".format(date))
  f.write("<ul>")
  for result in results["results"]["bindings"]:
    if ("scientist" in result):
      # create a wikipedia link
      wikiurl = "http://en.wikipedia.org/wiki/" + result["scientist"]["value"].split("/")[-1]
    else:
      wikiurl = "NONE"
    if ("name" in result):
      name = result["name"]["value"]
    else:
      name = "NONE"
    if ("birthdate" in result):
      birthdate = result["birthdate"]["value"]
    else:
      birthdate = 'NONE'
    if ("description" in result):
      description = result["description"]["value"]
    else:
      description =""
    if ("thumbnail" in result):
      pic = result["thumbnail"]["value"]
    else:
      pic = 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Question_mark2.svg/71px-Question_mark2.svg.png'
    #parse date as datetime
    dt = datetime.strptime(birthdate, '%Y-%m-%d')
    f.write('<li><b>{}</b> -- <img src="{}" height="60px"> <a href="{}">{}</a>, {} </li>'.format(dt.year, pic.replace("300", "60"), wikiurl, name, description))
  f.write("<ul>")
  f.write("</body></html>")
files.download("birthday.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>